---
---
---
# ***Recurrent Neural Network, RNN***
---
---
---

1. *Recurrent Neural Network, RNN*
2. *Long Short-Term Memory, LSTM*
3. *Gated Recurrent Unit, GRU*
4. *SimpleRNN과 LSTM*
5. *Recurrent Neural Network Language Model, RNNLM*
6. *Text Generation using RNN*

---
## 1. ***Recurrent Neural Network, RNN***
---

 - RNN은 대표적인 sequence model
 - DNN은 Feed Forward Neural Network 구조지만 RNN의 경우 Recurrent Neural Network 구조를 가짐
 - 즉, 출력층으로만 향하는 기존의 신경망과 달리 RNN은 출력층의 값이 다시 은닉층 노드로 향하는 순환신경망 구조


<br>

#### ***1. RNN***

 - RNN에서의 cell이란 ?
     - 은닉층에서 활성화 함수를 통해 결과를 내보내는 역할을 하는 노드
     - 이 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역할을 수행하므로 이를 메모리 셀 또는 RNN 셀이라고 표현
     - 은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 하고 있음
     - 메모리 셀이 출력층 방향으로 또는 다음 시점 t+1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 함

> RNN Structure <br>

![](https://wikidocs.net/images/page/22886/rnn_image2_ver3.PNG) <br>

> Detailed Structure <br>

![](https://wikidocs.net/images/page/22886/rnn_image2.5.PNG) <br>

> RNN Equation <br>

![](https://wikidocs.net/images/page/22886/rnn_image4_ver2.PNG)<br>
은닉층 : $h_t = tanh(w_x x_t + w_h h_{t-1} + b)$ <br>
출력층 : $y_t = f(w_y h_t + b)$

> RNN Input Shape <br>

![](https://wikidocs.net/images/page/22886/rnn_image6between7.PNG)

- How to print output state from cell?
    1. 메모리 셀의 최종 시점의 은닉 상태만을 리턴하고자 한다면 (batch_size, output_dim) 크기의 2D 텐서를 리턴
    2. 메모리 셀의 각 시점(time step)의 은닉 상태값들을 모아서 전체 시퀀스를 리턴하고자 한다면 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴
        - using `return_sequences = True`

In [1]:
# 함수 사용 없이 의사코드로 RNN 구현하기
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.
# print(hidden_state_t) 

Wx = np.random.random((hidden_size, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.87137823 0.96849597 0.92462722 0.98108858 0.85341294 0.91668457
  0.96730208 0.79581152]
 [0.99970633 0.99991227 0.99996763 0.99950346 0.99974216 0.99925189
  0.99972503 0.99976492]
 [0.99985349 0.99994016 0.99998157 0.99977212 0.99983561 0.99965979
  0.99988174 0.99991575]
 [0.99993806 0.99998953 0.99999255 0.9999555  0.99997966 0.99990359
  0.99998081 0.99995623]
 [0.99992786 0.99998231 0.99999437 0.99992596 0.99997286 0.99984459
  0.99997404 0.9999584 ]
 [0.99994168 0.99999403 0.99999624 0.99993958 0.99996639 0.99987995
  0.99996377 0.99995486]
 [0.99992397 0.99998967 0.99999375 0.99987852 0.99997787 0.99982952
  0.99992822 0.9999152 ]
 [0.99990211 0.99996941 0.99999221 0.99985981 0.99994747 0.99975539
  0.99994473 0.99994252]
 [0.99995598 0.99999541 0.99999712 0.99996143 0.99999211 0.99991691
  0.99998413 0.99996154]
 [0.99989453 0.99997007 0.99998958 0.99985916 0.99992035 0.99976239
  0.99993181 0.99993547]

<br>

#### ***2. Bidirectional Recurrent Neural Network***

 - 시점 t에서의 출력값을 예측할 때 이전 시점의 데이터뿐만 아니라, 이후 데이터에도 영향을 받을 수 있는 관점이다.
 - 과거 시점의 데이터만 고려하는 것이 아니라 향후 시점의 데이터에서도 힌트를 찾는다.
 - 양방향 RNN은 기본적으로 두개의 메모리 셀을 사용한다.
     1. 앞 시점의 은닉 상태를 전달받아 현재의 은닉 상태를 계산하는 셀. (Forward States)
     2. 뒤 시점의 은닉 상태를 전달받아 현재의 은닉 상태를 계산하는 셀. (Backward States)
     
     
![](https://wikidocs.net/images/page/22886/rnn_image6_ver3.PNG)
*주황색 cell이 Forward States를 받으며 초록색 cell이 Backward States를 받는다*

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional

timesteps = 2
input_dim = 10

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.summary()

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 2, 256)            35584     
Total params: 35,584
Trainable params: 35,584
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 2, 256)            35584     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 2, 256)            98560     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 2, 256)            98560     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 2, 256)            98560     
Total params: 331,264
Traina


> EXAMPLE

1. Embedding을 사용하며, 단어 집합(Vocabulary)의 크기가 5,000이고 임베딩 벡터의 차원은 100입니다.
2. 은닉층에서는 Simple RNN을 사용하며, 은닉 상태의 크기는 128입니다.
3. 훈련에 사용하는 모든 샘플의 길이는 30으로 가정합니다.
4. 이진 분류를 수행하는 모델로, 출력층의 뉴런은 1개로 시그모이드 함수를 사용합니다.
5. 은닉층은 1개입니다.


> RESULT

 - embedding = 5000 x 100 = 500000
 - wx = 100 x 128 = 12800
 - wh = 128 x 128 = 16384
 - h = 128
 - wy = 128
 - output = 1

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense

vocab_size = 5000
embedding_dim = 100
hidden_size = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_size))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         500000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               29312     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 529,441
Trainable params: 529,441
Non-trainable params: 0
_________________________________________________________________


---
## 2. ***Long Short-Term Memory, LSTM***
---

<br>

#### ***1. Vanila RNN***
 
![](https://wikidocs.net/images/page/22888/lstm_image1_ver2.PNG) <br>
- vanila RNN은 비교적 짧은 시퀀스에 대해서만 효과를 보인다.
- timestep이 길어질 수록 앞의 정보가 충분히 전달되지 못하는 현상이 발생한다.
- 장기 의존성 문제 (The Problem of Long-Term Dependencies)를 가진다.

<br>

#### ***2. LSTM***

> RNN과 LSTM의 내부 구조 비교

![](https://wikidocs.net/images/page/22888/vanilla_rnn_ver2.PNG) 
![](https://wikidocs.net/images/page/22888/vaniila_rnn_and_different_lstm_ver2.PNG) <br>

- 기존 은닉층의 메모리 셀에 **입력게이터, 망각게이트, 출력게이트**를 추가
- 불필요한 기억을 지우고, 기억해야할 것들을 정함
- hidden state와 cell state를 전달하게됨


---
## 3. ***Gated Recurrent Unit, GRU***
---

- LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄임
- 즉, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화


![](https://wikidocs.net/images/page/22889/GRU.PNG) <br>
 - 기존 LSTM은 입력, 망각, 출력 게이트가 존재
 - GRU에서는 업데이트, 리셋 두개의 게이트를 사용
 
<br>
 
>실제 GRU 은닉층을 추가하는 코드. <br>
model.add(GRU(hidden_size, input_shape=(timesteps, input_dim)))

---
## 4. ***SimpleRNN과 LSTM***
---

> **example** <br>
dim = 5 <br>
timestep(length) = 4 <br>
batch size = 1

In [1]:
# !pip3 install wrapt
import numpy as np    
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

# dim = 5, length(ts) = 4, bs = 1
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)



<br>

#### ***1. Simple RNN 구현***

1. SimpleRNN(default)
2. SimpleRNN(retrun_sequences=True)
3. SimpleRNN(return_sequences=True, return_state=True)
    - output1 : hidden state at all timestep
    - output2 : hidden state at last timestep

In [9]:
# 1. SimpleRNN(default)
rnn = SimpleRNN(3)
# rnn = SimpleRNN(3, return_sequences=False, return_state=False)와 동일.
hidden_state = rnn(train_X)

# 최종  output : 마지막 timestep에서의 출력값
print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.94330335 -0.97819847 -0.8401196 ]], shape: (1, 3)


In [10]:
# 2. SimpleRNN(retrun_sequences=True)
rnn = SimpleRNN(3, return_sequences=True)
hidden_states = rnn(train_X)

# 최종 output : 모든 timestep에서의 출력값
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[-0.9938013  -0.82615274 -0.47604564]
  [-0.99873894 -0.66273797 -0.1694383 ]
  [-0.99903196  0.8662755  -0.983821  ]
  [-0.03076643 -0.9436481  -0.8706647 ]]], shape: (1, 4, 3)


In [4]:
# 3. SimpleRNN(return_sequences=True, return_state=True)
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_state, last_state.shape))

hidden states : [[[-0.72086656  0.9934091  -0.953686  ]
  [ 0.19770178  0.8997607  -0.976504  ]
  [ 0.42180285  0.9290261  -0.09734499]
  [-0.9015882   0.20464553 -0.6407577 ]]], shape: (1, 4, 3)
last hidden state : [[-0.9015882   0.20464553 -0.6407577 ]], shape: (1, 3)



<br>

#### ***2. LSTM 구현***
1. LSTM(default)
2. LSTM(retrun_sequences=True)
3. LSTM(return_sequences=True, return_state=True)
    - output1 : hidden state at all timestep
    - output2 : hidden state at last timestep
    - output3 : cell state at last timestep

In [9]:
# 1. LSTM(default)
lstm = LSTM(3)
hidden_state = lstm(train_X)

# 최종  output : 마지막 timestep에서의 출력값
print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[ 0.07909779 -0.05342373 -0.03087045]], shape: (1, 3)


In [11]:
# 2. LSTM(retrun_sequences=True)
lstm = LSTM(3, return_sequences=True)
hidden_states = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))

hidden states : [[[-0.04998615 -0.4816332  -0.09267705]
  [-0.0082723  -0.705354   -0.3168589 ]
  [ 0.29138252 -0.50929123 -0.31556296]
  [ 0.08178879 -0.4883236  -0.10235637]]], shape: (1, 4, 3)


In [12]:
# 3. LSTM(retrun_sequences=True, return_state=True)
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_hidden_state, last_cell_state = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last hidden state : {}, shape: {}'.format(last_hidden_state, last_hidden_state.shape))
print('last cell state : {}, shape: {}'.format(last_cell_state, last_cell_state.shape))

hidden states : [[[-0.05346462 -0.14736414 -0.55206156]
  [-0.08086106 -0.24957754 -0.5361125 ]
  [-0.1453886  -0.24032782 -0.25564075]
  [ 0.00431521 -0.27720132 -0.23613769]]], shape: (1, 4, 3)
last hidden state : [[ 0.00431521 -0.27720132 -0.23613769]], shape: (1, 3)
last cell state : [[ 0.00641178 -0.4084831  -0.41746327]], shape: (1, 3)



<br>

#### ***3. Bidirectional(LSTM) 구현***


In [2]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

In [3]:
# 1. Bidirectional(retrun_sequences=False, return_state=True)
bilstm = Bidirectional(LSTM(3, return_sequences=False, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


In [4]:
# 1. Bidirectional(retrun_sequences=True, return_state=True)
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, \
                            kernel_initializer=k_init, bias_initializer=b_init, recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)
